# Variantional AutoEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
os.chdir("./drive/My Drive/Colab Notebooks/pytorch")

In [ ]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# creat a directory if not exists
sample_dir = 'samples_vae'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [ ]:
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../../data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

In [ ]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h) # returns mu and log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)  # add gaussian normal noise
        return mu + eps * std
    
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [ ]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)

        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print ("Epoch [{}/{}], Step [{}/{}], Recent Loss: {:.4f}, KL Div: {:.4f}".format(
                epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()
            ))
    
    if epoch % 2 == 1:
        with torch.no_grad():
            # Save the sampled images
            z = torch.randn(batch_size, z_dim).to(device)
            out = model.decode(z).view(-1, 1, 28, 28)
            save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

            # Save the reconstructed images
            out, _, _ = model(x)
            x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
            save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch [1/15], Step [10/469], Recent Loss: 36095.8906, KL Div: 2861.3711
Epoch [1/15], Step [20/469], Recent Loss: 29923.4668, KL Div: 921.9568
Epoch [1/15], Step [30/469], Recent Loss: 27725.2344, KL Div: 1132.2158
Epoch [1/15], Step [40/469], Recent Loss: 27098.5391, KL Div: 700.8206
Epoch [1/15], Step [50/469], Recent Loss: 27032.6836, KL Div: 665.4869
Epoch [1/15], Step [60/469], Recent Loss: 25868.7617, KL Div: 747.5256
Epoch [1/15], Step [70/469], Recent Loss: 25921.4590, KL Div: 798.6056
Epoch [1/15], Step [80/469], Recent Loss: 24955.8535, KL Div: 870.9413
Epoch [1/15], Step [90/469], Recent Loss: 23548.1445, KL Div: 1119.0024
Epoch [1/15], Step [100/469], Recent Loss: 23026.0410, KL Div: 1219.3431
Epoch [1/15], Step [110/469], Recent Loss: 22012.3301, KL Div: 1350.2778
Epoch [1/15], Step [120/469], Recent Loss: 21504.8047, KL Div: 1408.7236
Epoch [1/15], Step [130/469], Recent Loss: 20476.6895, KL Div: 1384.8651
Epoch [1/15], Step [140/469], Recent Loss: 19053.2109, KL Div: 160